# Import

In [ ]:
import tensorflow as tf
from tensorflow import keras
!pip install -q -U keras-tuner
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch, BayesianOptimization 

     |████████████████████████████████| 98 kB 3.0 MB/s 


# Loading Data

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
# Normalize pixel values between (img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.00 
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

4431872/4422102 [==============================] - 0s 0us/step


# Base Model

In [ ]:
def model_builder():
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units=512, activation='relu'))
  model.add(keras.layers.Dense(10))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
model = model_builder()
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)


Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5896 - accuracy: 0.7848 - val_loss: 0.4686 - val_accuracy: 0.8259
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4261 - accuracy: 0.8447 - val_loss: 0.4282 - val_accuracy: 0.8407
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3833 - accuracy: 0.8597 - val_loss: 0.4037 - val_accuracy: 0.8534
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3558 - accuracy: 0.8691 - val_loss: 0.3841 - val_accuracy: 0.8593
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3379 - accuracy: 0.8745 - val_loss: 0.3758 - val_accuracy: 0.8597
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3191 - accuracy: 0.8820 - val_loss: 0.3457 - val_accuracy: 0.8731
Epoch 7/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3087 - accuracy: 0.8842 - val_loss: 0.3465 - val_accuracy:

# Keras Tuner

as it wrote in the doc, number of hidden neurons should be between 16 and 512 with step size = 16

and also learning rate must be one of 0.01, 0.0001, 0.0005, 0.005



In [ ]:
def create_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    model.add(keras.layers.Dense(units=hp.Int('dense',
                                              min_value = 16,
                                              max_value = 512,
                                              step = 16)
                                 , activation='relu'))
    
    model.add(keras.layers.Dense(10))
    learning_rates = [5e-3, 5e-4, 1e-2, 1e-3, 1e-4]
    opt = keras.optimizers.Adam(learning_rate =hp.Choice('lr', values= learning_rates))
    model.compile(optimizer=opt,
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model

In [ ]:
!rm -rf /content/fashion_mnist

In [ ]:
tuner = BayesianOptimization(
        create_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='fashion_mnist',
        project_name='KerasTuner')

In [ ]:
NUM_EPOCH = 50
tuner.search(x=img_train,
             y=label_train,
             epochs=NUM_EPOCH,
             batch_size=128,
             validation_split= 0.2)

print('**' * 50)
print(tuner.get_best_models()[0].summary())
print(tuner.get_best_hyperparameters()[0].values)

Trial 10 Complete [00h 01m 22s]
val_accuracy: 0.8933333158493042

Best val_accuracy So Far: 0.8974999785423279
Total elapsed time: 00h 13m 40s
INFO:tensorflow:Oracle triggered exit
****************************************************************************************************
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
None
{'dense': 512, 'lr': 0.005}

In [ ]:
# get the best model and evalute it on the test data
model = tuner.get_best_models(num_models=1)[0]
print (model.summary())
# Evaluate the best model.
loss, accuracy = model.evaluate(img_test, label_test)
print('loss Test:', loss)
print('accuracy Test:', accuracy)
model.save('prueba1.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
None
313/313 [==============================] - 1s 3ms/step - loss: 0.3381 - accuracy: 0.8888
loss Test: 0.33807000517845154
accuracy Test: 0.8888000249862671


In [ ]:
loss, accuracy = model.evaluate(img_train, label_train)
print('loss Train:', loss)
print('accuracy Train:', accuracy)

1875/1875 [==============================] - 5s 3ms/step - loss: 0.1856 - accuracy: 0.9349
loss Train: 0.18558098375797272
accuracy Train: 0.9348999857902527
